# Evaluate 
$ f(x, y) = 2x^2 - x^2y^2 + 3 $

$ g_0 = x ⋅ x $ 

$ g_1 = x ⋅ x $ 

$ g_2 = y ⋅ y $

$ g_3 = g_0 ⋅ 2 $

$ g_4 = g_1 ⋅ g_2 $

$ g_5 = g_2 - g_4 $

$ g_6 = g_5 + 3 $

---

$ c_0 = a_0 ⋅ b_0 $

$ c_1 = a_1 ⋅ b_1 $

$ c_2 = a_2 ⋅ b_2 $

$ c_3 = a_3 ⋅ 2 $

$ c_4 = a_4 ⋅ b_4 $

$ c_5 = a_5 - b_5 $

$ c_6 = a_6 + 3 $

---

$ x = a_0 = b_0 = a_1 = b_1 $

$ y = a_2 = b_2 $

$ g_0 = a_3 = c_0 $

$ g_1 = a_4 = c_1 $

$ g_2 = b_4 = c_2 $

$ g_3 = a_5 = c_3 $

$ g_4 = b_5 = c_4 $

$ g_5 = a_6 = c_5 $

$ g_6 = c_6 $

### Gate constrain equation

$ g_i = q_{Li}⋅a_i+q_{Ri}⋅b_i+q_{Mi}⋅(a_i⋅b_i)+q_{Ci}+q_{Oi}⋅c_i=0 $

$ g_0 = 0⋅a_0 + 0⋅b_0 + 1⋅(a_0⋅b_0) + 0 + (-1)⋅c_0 = 0 $

$ g_1 = 0⋅a_1 + 0⋅b_1 + 1⋅(a_1⋅b_1) + 0 + (-1)⋅c_1 = 0 $

$ g_2 = 0⋅a_2 + 0⋅b_2 + 1⋅(a_2⋅b_2) + 0 + (-1)⋅c_2 = 0 $

$ g_3 = 2⋅a_3 + 0⋅b_3 + 0⋅(a_3⋅b_3) + 0 + (-1)⋅c_3 = 0 $

$ g_4 = 0⋅a_4 + 0⋅b_4 + 1⋅(a_4⋅b_4) + 0 + (-1)⋅c_4 = 0 $

$ g_5 = 1⋅a_5 + (-1)⋅b_5 + 0⋅(a_5⋅b_5) + 0 + (-1)⋅c_5 = 0 $

$ g_6 = 1⋅a_6 + 0⋅b_6 + 0⋅(a_6⋅b_6) + 3 + (-1)⋅c_6 = 0 $

**selector vectors**

$ g_i = [q_{Li}, q_{Ri}, q_{Mi}, q_{Ci}, q_{Oi}] $

$ g_0 = [0, 0, 1, 0, -1] $

$ g_1 = [0, 0, 1, 0, -1] $

$ g_2 = [0, 0, 1, 0, -1] $

$ g_3 = [2, 0, 0, 0, -1] $

$ g_4 = [0, 0, 1, 0, -1] $

$ g_5 = [1, -1, 0, 0, -1] $

$ g_6 = [1, 0, 0, 3, -1] $

**let x = 2 y = 3**


| Vectors| a | b | c | qLi | qRi | qMi | qCi | qOi |
| --- | --- | --- | --- |--- | --- | --- | --- | --- |
| gate0 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate1 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate2 | 3 | 3 | 9 | 0 | 0 | 1 | 0 | -1 |
| gate3 | 4 | 0 | 8 | 2 | 0 | 0 | 0 | -1 |
| gate4 | 4 | 9 | 36 | 0 | 0 | 1 | 0 | -1 |
| gate5 | 8 | 36 | -28 | 1 | -1 | 0 | 0 | -1 |
| gate6 | -28 | 3 | -25 | 1 | 0 | 0 | 3 | -1 |

In [1]:
import galois
import numpy as np
from py_ecc.optimized_bls12_381 import add, multiply, G1, G2, neg, pairing, eq

x = 2
y = 3

out = 2*x**2 - x**2*y**2 + 3
print(out)

a = [2, 2, 3, 4, 4, 8, -28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, -28, -25]

ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, -1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1, -1]
qpi = [0, 0, 0, 0, 0, 0, 0]

def to_galois_array(a, field):
    # normalize to positive values
    a = [x % field.order for x in a]
    return GF(a)

def to_vanishing_poly(roots, field):
    # Z^n - 1 = (Z - 1)(Z - w)(Z - w^2)...(Z - w^(n-1))
    return galois.Poly.Degrees([len(roots), 0], coeffs=[1, -1], field=field)

def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if type(v) == list else v
    return galois.lagrange_poly(x, y)

def pad_array(a, n):
    return a + [0]*(n - len(a))

n = len(a)
# we have 7 gates, but we need to pad to 2^k
n = 2**int(np.ceil(np.log2(n)))

# we have to pad the arrays to the same length
a = pad_array(a, n)
b = pad_array(b, n)
c = pad_array(c, n)
ql = pad_array(ql, n)
qr = pad_array(qr, n)
qm = pad_array(qm, n)
qc = pad_array(qc, n)
qo = pad_array(qo, n)
qpi = pad_array(qpi, n)

p = 241
GF = galois.GF(p)
w = GF.primitive_root_of_unity(n)
assert w**n == 1
tau = GF(12)

roots = GF([w**x for x in range(0, n)])

_A = to_poly(roots, a, GF)
_B = to_poly(roots, b, GF)
_C = to_poly(roots, c, GF)
ZH = to_vanishing_poly(roots, GF)
for x in roots:
    assert ZH(x) == 0

# a bit awkward, but we need to generate random values in GF for blinding polynomials
# b1, .., b9
b_random = galois.Poly.Random(degree=10, field=GF).coeffs

bA = galois.Poly(b_random[:2], field=GF)
bB = galois.Poly(b_random[2:4], field=GF)
bC = galois.Poly(b_random[4:6], field=GF)

A = _A + bA*ZH
B = _B + bB*ZH
C = _C + bC*ZH

QL = to_poly(roots, ql, GF)
QR = to_poly(roots, qr, GF)
QM = to_poly(roots, qm, GF)
QC = to_poly(roots, qc, GF)
QO = to_poly(roots, qo, GF)
QPI = to_poly(roots, qpi, GF)

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + qc(x) + c(x)*qo(x)
G = A*QL + B*QR + A*B*QM + QC + C*QO + QPI

print("--- Vanishing Polynomial ---")
print("ZH = ", ZH)

print("\n\n--- Blinding Polynomials ---")
print(f"bA = {bA}")
print(f"bB = {bB}")
print(f"bC = {bC}")

print("\n\n--- Wire Polynomials ---")
print(f"A = {_A}")
print(f"B = {_B}")
print(f"C = {_C}")

print("\n\n--- Blinded Wire Polynomials ---")
print(f"A' = {A}")
print(f"B' = {B}")
print(f"C' = {C}")

print("\n\n--- Selector Polynomials ---")
print(f"QL = {QL}")
print(f"QR = {QR}")
print(f"QM = {QM}")
print(f"QC = {QC}")
print(f"QO = {QO}")
print(f"QPI = {QPI}")

print("\n\n--- Gate Constraints Polynomial ---")
print(f"G = {G}")
for i in range(0, len(roots)):
    print(f"gate #{i} G({roots[i]}) = {G(roots[i])} --> {'OK' if G(roots[i]) == 0 else 'FAIL'}")

assert G % ZH == 0

# TODO: calculate [A], [B], [C]
round1 = [A(tau), B(tau), C(tau)]
print("\n\n--- Round 1 ---")
print(f"Round 1 = {round1}")

-25


OMP: Info #270: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


--- Vanishing Polynomial ---
ZH =  x^8 + 240


--- Blinding Polynomials ---
bA = 17x + 79
bB = 112x + 223
bC = 198x + 127


--- Wire Polynomials ---
A = 155x^7 + 227x^6 + 76x^5 + 26x^4 + 192x^3 + 82x^2 + 58x + 150
B = 203x^7 + 28x^6 + 176x^5 + 148x^4 + 217x^3 + 154x^2 + 3x + 37
C = 93x^7 + 22x^6 + 89x^5 + 5x^4 + 78x^3 + 233x^2 + 206x + 1


--- Blinded Wire Polynomials ---
A' = 17x^9 + 79x^8 + 155x^7 + 227x^6 + 76x^5 + 26x^4 + 192x^3 + 82x^2 + 41x + 71
B' = 112x^9 + 223x^8 + 203x^7 + 28x^6 + 176x^5 + 148x^4 + 217x^3 + 154x^2 + 132x + 55
C' = 198x^9 + 127x^8 + 93x^7 + 22x^6 + 89x^5 + 5x^4 + 78x^3 + 233x^2 + 8x + 115


--- Selector Polynomials ---
QL = 187x^7 + 38x^6 + 119x^5 + 60x^4 + 70x^3 + 22x^2 + 106x + 121
QR = 64x^7 + 8x^6 + x^5 + 211x^4 + 177x^3 + 233x^2 + 240x + 30
QM = 57x^7 + 211x^6 + 73x^5 + 211x^4 + 168x^3 + 211x^2 + 184x + 91
QC = 24x^7 + 90x^6 + 217x^5 + 151x^4 + 24x^3 + 90x^2 + 217x + 151
QO = 240x^7 + 8x^6 + 177x^5 + 30x^4 + x^3 + 233x^2 + 64x + 210
QPI = 0


--- Gate Con

### Permutation constraint

In [2]:
import sha3


def numbers_to_hash(numbers: [int], field) -> int:
    """Hash a number."""
    return field(int(sha3.keccak_256(bytes(numbers)).hexdigest(), 16) % field.order)

def print_sigma(sigma, a, b, c, r):
    group_size = len(sigma) // 3
    padding = 6

    print(f"{' w'} | {'value':{padding}} | {'i':{padding}} | {'sigma(i)':{padding}}")

    for i in range(0, group_size):
        print(f"a{i} | {a[i]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(group_size, 2 * group_size):
        print(f"b{i - group_size} | {b[i - group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(2 * group_size, 3 * group_size):
        print(f"c{i - 2 * group_size} | {c[i - 2 * group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

# TODO: calculate beta, gamma from [A], [B], [C]
beta = numbers_to_hash(round1 + [0], GF)
gamma = numbers_to_hash(round1 + [1], GF)

ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[0], ai[1]: ai[1], ai[2]: ai[2], ai[3]: ci[0], ai[4]: ci[1], ai[5]: ci[3], ai[6]: ci[5], ai[7]: ai[7],
    bi[0]: bi[0], bi[1]: bi[1], bi[2]: bi[2], bi[3]: bi[3], bi[4]: ci[2], bi[5]: ci[4], bi[6]: bi[6], bi[7]: bi[7],
    ci[0]: ai[3], ci[1]: ai[4], ci[2]: bi[4], ci[3]: ai[5], ci[4]: bi[5], ci[5]: ai[6], ci[6]: ci[6], ci[7]: ci[7],
}

k1 = 2
k2 = 4
c1_roots = roots
c2_roots = roots * k1
c3_roots = roots * k2

c_roots = np.concatenate((c1_roots, c2_roots, c3_roots))

check = set()
for r in c_roots:
    if int(r) in check:
        print(f"Duplicate root {r} in {c_roots}")
        assert False
    check.add(int(r))

sigma1 = GF([c_roots[sigma[i]] for i in range(0, n)])
sigma2 = GF([c_roots[sigma[i + n]] for i in range(0, n)])
sigma3 = GF([c_roots[sigma[i + 2 * n]] for i in range(0, n)])

print_sigma(sigma, a, b, c, c_roots)

print("\n\n--- Cosest ---")
print(f"c0 = {c1_roots}")
print(f"c1 = {c2_roots}")
print(f"c2 = {c3_roots}")

print("\n\n--- Sigma ---")
print(f"sigma1 = {sigma1}")
print(f"sigma2 = {sigma2}")
print(f"sigma3 = {sigma3}")

S1 = to_poly(roots, sigma1, GF)
S2 = to_poly(roots, sigma2, GF)
S3 = to_poly(roots, sigma3, GF)

I1 = to_poly(roots, c1_roots, GF)
I2 = to_poly(roots, c2_roots, GF)
I3 = to_poly(roots, c3_roots, GF)

padding = 3
for i in range(0, len(roots)):
    s = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"   A({roots[i]:{padding}}) = {A(roots[i]):{padding}} "
    s += f"   B({roots[i]:{padding}}) = {B(roots[i]):{padding}} "
    s += f"   C({roots[i]:{padding}}) = {C(roots[i]):{padding}} "
    s += f"   I1({roots[i]:{padding}}) = {I1(roots[i]):{padding}} "
    s += f"   I2({roots[i]:{padding}}) = {I2(roots[i]):{padding}} "
    s += f"   I3({roots[i]:{padding}}) = {I3(roots[i]):{padding}} "
    s += f"   S1({roots[i]:{padding}}) = {S1(roots[i]):{padding}} "
    s += f"   S2({roots[i]:{padding}}) = {S2(roots[i]):{padding}} "
    s += f"   S3({roots[i]:{padding}}) = {S3(roots[i]):{padding}} "
    print(s)


f = (A + I1 * beta + gamma) * (B + I2 * beta + gamma) * (C + I3 * beta + gamma)
g = (A + S1 * beta + gamma) * (B + S2 * beta + gamma) * (C + S3 * beta + gamma)

print("\n\n--- f(x) ---")
print(f"f(x) = {f}")

print("\n\n--- g(x) ---")
print(f"g(x) = {g}")

acc_poly_eval = [GF(1)]
for i in range(0, n):
    acc_poly_eval.append(
        acc_poly_eval[-1] * f(roots[i]) / g(roots[i])
    )

assert acc_poly_eval.pop() == GF(1)
ACC = galois.lagrange_poly(roots, GF(acc_poly_eval))

print("\n\n--- Accumulator Polynomial ---")
print(f"ACC(x) = {ACC}")

# blinding polynomial for accumulator
bZ = galois.Poly(b_random[6:9], field=GF)
print(f"bZ = {bZ}")
Z = bZ * ZH + ACC

print("\n\n--- Z ---")
print(f"Z(x) = {Z}")

for r in roots:
    print(f"Z({r}) = {Z(r)}")

assert Z(roots[0]) == 1
assert Z(roots[-1]) == 1

round2 = [Z(tau)]
print("\n\n--- Round 2 ---")
print(f"Round 2 = {round2}")

 w | value  | i      | sigma(i)
a0 |      2 |      1 |      1
a1 |      2 |     30 |     30
a2 |      3 |    177 |    177
a3 |      4 |      8 |      4
a4 |      4 |    240 |    120
a5 |      8 |    211 |     32
a6 |    -28 |     64 |    121
a7 |      0 |    233 |    233
-- | --     | --     | --    
b0 |      2 |      2 |      2
b1 |      2 |     60 |     60
b2 |      3 |    113 |    113
b3 |      0 |     16 |     16
b4 |      9 |    239 |    226
b5 |     36 |    181 |    237
b6 |      3 |    128 |    128
b7 |      0 |    225 |    225
-- | --     | --     | --    
c0 |      4 |      4 |      8
c1 |      4 |    120 |    240
c2 |      9 |    226 |    239
c3 |      8 |     32 |    211
c4 |     36 |    237 |    181
c5 |    -28 |    121 |     64
c6 |    -25 |     15 |     15
c7 |      0 |    209 |    209


--- Cosest ---
c0 = [  1  30 177   8 240 211  64 233]
c1 = [  2  60 113  16 239 181 128 225]
c2 = [  4 120 226  32 237 121  15 209]


--- Sigma ---
sigma1 = [  1  30 177   4 120  32 121 

In [3]:
# TODO: calculate [Z]
alpha = numbers_to_hash(round1 + round2, GF)

def eval_poly(poly: galois.Poly, domain, shift=GF(1)):
    coeff = poly.coeffs[::-1]
    eval = []
    for j in range(len(domain)):
        eval += [sum([(domain[j] * shift) ** i * coeff[i] for i in range(poly.degree+1)], GF(0))]
    return GF(eval)

def shift_poly(poly: galois.Poly, domain):
    shift = domain[0] if domain[0] != GF(1) else domain[1]
    shifted_eval = eval_poly(poly, domain, shift=shift)
    return galois.lagrange_poly(domain, shifted_eval)

L1 = galois.lagrange_poly(roots, GF([1] + [GF(0)] * (n - 1)))
for i, r in enumerate(roots):
    print(f"L1({r}) = {L1(r)}")
    assert L1(r) == (GF(1) if i == 0 else GF(0))

# Find Z offset
# roots[1] is a primitive root of unity and a generator of the subgroup
ZW = shift_poly(Z, roots)
for r in roots:
    print(f"Z({r:3}) = {Z(r):3} -> ZW({r:3}) = {ZW(r):3}")

# assert ZW * g == Z * f

T0 = G // ZH
assert G % ZH == 0

T1 = (f * Z - g * ZW) * alpha
assert T1 % ZH == 0

T2 = (Z - GF(1)) * L1 * alpha**2
assert T2 % ZH == 0

T = (T0 + T1 + T2) // ZH

print("\n\n--- T ---")
print(f"T(x) = {T}")

t_coeffs = T.coeffs[::-1]

Tl = galois.Poly(t_coeffs[:n][::-1], field=GF)
Tm = galois.Poly(t_coeffs[n:2*n][::-1], field=GF)
Th = galois.Poly(t_coeffs[2*n:][::-1], field=GF)

print("\n\n--- T' ---")
print(f"Tl(x) = {Tl}")
print(f"Tm(x) = {Tm}")
print(f"Th(x) = {Th}")

X_n = galois.Poly.Degrees([n, 0], coeffs=[1, 0], field=GF)
X_2n = galois.Poly.Degrees([2*n, 0], coeffs=[1, 0], field=GF)
# make sure that T was split correctly
# T = TL + X^n * TM + X^2n * TH
assert T == (Tl + X_n * Tm + X_2n * Th)

TL = Tl + b_random[8] * X_n
TM = Tm - b_random[8] + b_random[9] * X_n
TH = Th - b_random[9]

assert T == (TL + X_n * TM + X_2n * TH)

# TODO: calculate [TL], [TM], [TH]

round3 = [TL(tau), TM(tau), TH(tau)]
print("\n\n--- Round 3 ---")
print(f"Round 3 = {round3}")

L1(1) = 1
L1(30) = 0
L1(177) = 0
L1(8) = 0
L1(240) = 0
L1(211) = 0
L1(64) = 0
L1(233) = 0
Z(  1) =   1 -> ZW(  1) =  10
Z( 30) =  10 -> ZW( 30) =  52
Z(177) =  52 -> ZW(177) =  33
Z(  8) =  33 -> ZW(  8) =  38
Z(240) =  38 -> ZW(240) =  46
Z(211) =  46 -> ZW(211) = 132
Z( 64) = 132 -> ZW( 64) =   1
Z(233) =   1 -> ZW(233) =   1


--- T ---
T(x) = 11x^29 + 166x^28 + 13x^27 + 51x^26 + 221x^25 + 13x^24 + 189x^23 + 137x^22 + 213x^21 + 169x^20 + 111x^19 + 43x^18 + 208x^17 + 14x^16 + 27x^15 + 29x^14 + 54x^13 + 213x^12 + 140x^11 + 193x^10 + 129x^9 + 56x^8 + 184x^7 + 205x^6 + 105x^5 + 128x^4 + 47x^3 + 84x^2 + 38x + 178


--- T' ---
Tl(x) = 184x^7 + 205x^6 + 105x^5 + 128x^4 + 47x^3 + 84x^2 + 38x + 178
Tm(x) = 27x^7 + 29x^6 + 54x^5 + 213x^4 + 140x^3 + 193x^2 + 129x + 56
Th(x) = 11x^13 + 166x^12 + 13x^11 + 51x^10 + 221x^9 + 13x^8 + 189x^7 + 137x^6 + 213x^5 + 169x^4 + 111x^3 + 43x^2 + 208x + 14


--- Round 3 ---
Round 3 = [GF(184, order=241), GF(143, order=241), GF(112, order=241)]


In [4]:
# TODO: calculate zeta from [TL], [TM], [TH]
zeta = numbers_to_hash(round1 + round2 + round3, GF)

A_zeta = A(zeta)
B_zeta = B(zeta)
C_zeta = C(zeta)
S1_zeta = S1(zeta)
S2_zeta = S2(zeta)
ZW_zeta = ZW(zeta)

print("Openings:")
print(f"A(zeta) = {A_zeta}")
print(f"B(zeta) = {B_zeta}")
print(f"C(zeta) = {C_zeta}")
print(f"S1(zeta) = {S1_zeta}")
print(f"S2(zeta) = {S2_zeta}")
print(f"Zw(zeta) = {ZW_zeta}")

round4 = [A_zeta, B_zeta, C_zeta, S1_zeta, S2_zeta, ZW_zeta]
print("\n\n--- Round 4 ---")
print(f"Round 4 = {round4}")

Openings:
A(zeta) = 102
B(zeta) = 232
C(zeta) = 88
S1(zeta) = 226
S2(zeta) = 64
Zw(zeta) = 191


--- Round 4 ---
Round 4 = [GF(102, order=241), GF(232, order=241), GF(88, order=241), GF(226, order=241), GF(64, order=241), GF(191, order=241)]


In [5]:
# TODO compute v from all the values
# opening challenge v
v = numbers_to_hash(round1 + round2 + round3 + round4, GF)

# Compute linearisation polynomial r(X)
R0 = A_zeta * B_zeta * QM + A_zeta * QL + B_zeta * QR + C_zeta * QO + QC + QPI(zeta)

R1 = alpha * ((A_zeta + beta * zeta + gamma)*(B_zeta + beta * k1 * zeta + gamma) * (C_zeta + beta * k2 * zeta + gamma) * Z \
    - (A_zeta + beta * S1_zeta + gamma)*(B_zeta + beta * S2_zeta + gamma) * (C_zeta + beta * S3 + gamma) * ZW_zeta)

R2 = alpha**2 * ((Z - GF(1)) * L1(zeta))

R3 = ZH(zeta) * (TL + zeta**n * TM + zeta**(2*n) * TH)

R = R0 + R1 + R2 - R3

print("\n\n--- R ---")
print(f"R(x) = {R}")

# Compute opening proof polynomial Wz(X)

X_minus_zeta = galois.Poly([1, -zeta], field=GF)
print(f"X - zeta = {X_minus_zeta}")

W_zeta = R + \
    v * (A - A_zeta) + \
    v**2 * (B - B_zeta) + \
    v**3 * (C - C_zeta) + \
    v**4 * (S1 - S1_zeta) + \
    v**5 * (S2 - S2_zeta)

W_zeta = W_zeta // X_minus_zeta

assert w == (roots[1] if roots[0] == GF(1) else roots[0])
# Compute opening proof polynomial Wzω(X)
X_minus_zeta_omega = galois.Poly([1, -zeta * w], field=GF)

W_zeta_omega = (Z - ZW_zeta) // X_minus_zeta_omega

print("\n\n--- W ---")
print(f"Wz(x) = {W_zeta}")
print(f"Wzω(x) = {W_zeta_omega}")

# TODO: Compute [Wz] ,[Wzω]

round5 = [W_zeta(tau), W_zeta_omega(tau)]    

print("\n\n--- Round 5 ---")
print(f"Round 5 = {round5}")

# TODO: compute u from transcript
u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, GF)



--- R ---
R(x) = 79x^13 + 31x^12 + 181x^11 + 162x^10 + 234x^9 + 221x^8 + 58x^7 + 65x^6 + 13x^5 + 118x^4 + 157x^3 + 60x^2 + 7x + 67
X - zeta = x + 213


--- W ---
Wz(x) = 79x^12 + 74x^11 + 84x^10 + 104x^9 + 33x^8 + 221x^7 + 161x^6 + 194x^5 + 137x^4 + 92x^3 + 123x^2 + 59x + 154
Wzω(x) = 99x^9 + 230x^8 + 201x^7 + 100x^6 + 209x^5 + 184x^4 + 186x^3 + 238x^2 + 220x + 124


--- Round 5 ---
Round 5 = [GF(76, order=241), GF(208, order=241)]


# Verifier

In [6]:
# Preprocessed
_qm_t = QM(tau)
_ql_t = QL(tau)
_qr_t = QR(tau)
_qo_t = QO(tau)
_qc_t = QC(tau)
_s1_t = S1(tau)
_s2_t = S2(tau)
_s3_t = S3(tau)

In [7]:
_A_t, _B_t, _C_t = round1
_Z_t = round2[0]
_TL_t, _TM_t, _TH_t = round3
_Wz_t, _Wz_omega_t = round5

# round4 = [A_zeta, B_zeta, C_zeta, S1_zeta, S2_zeta, ZW_zeta]
_a_z, _b_z, _c_z, _s1_z, _s2_z, _zw_z = round4

_beta = numbers_to_hash(round1 + [0], GF)
_gamma = numbers_to_hash(round1 + [1], GF)
_alpha = numbers_to_hash(round1 + round2, GF)
_zeta = numbers_to_hash(round1 + round2 + round3, GF)
_v = numbers_to_hash(round1 + round2 + round3 + round4, GF)
_u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, GF)

assert _beta == beta
assert _gamma == gamma
assert _alpha == alpha
assert _zeta == zeta
assert _v == v
assert _u == u

assert _A_t == A(tau)
assert _B_t == B(tau)
assert _C_t == C(tau)
assert _Z_t == Z(tau)
assert _TL_t == TL(tau)
assert _TM_t == TM(tau)
assert _TH_t == TH(tau)
assert _Wz_t == W_zeta(tau)
assert _Wz_omega_t == W_zeta_omega(tau)

assert _a_z == A_zeta
assert _b_z == B_zeta
assert _c_z == C_zeta
assert _s1_z == S1_zeta
assert _s2_z == S2_zeta
assert _zw_z == ZW_zeta

print("\n\n -- Recovered Values --")
print(f"beta = {_beta}")
print(f"gamma = {_gamma}")
print(f"alpha = {_alpha}")
print(f"zeta = {_zeta}")
print(f"v = {_v}")
print(f"u = {_u}")



 -- Recovered Values --
beta = 71
gamma = 173
alpha = 165
zeta = 28
v = 102
u = 55


In [8]:
# Compute zero polynomial evaluation ZH(z) = z^n − 1.
ZH_zeta = ZH(zeta)

# Compute Lagrange polynomial evaluation L1(z) = ω(z^n−1) / n(z−ω)
L1_zeta = L1(zeta)

# Compute public input polynomial evaluation PI(z) = Pi∈[l] wiLi(z).
PI_zeta = QPI(zeta)

In [9]:
_r0 = PI_zeta - L1_zeta * alpha**2 - alpha * (_a_z + beta * _s1_z + gamma) * (_b_z + beta * _s2_z + gamma) * (_c_z + gamma) * _zw_z

_D0 = _a_z * _b_z * _qm_t + _a_z * _ql_t + _b_z * _qr_t + _c_z * _qo_t + _qc_t

_D11 = ((_a_z + beta * zeta + gamma) * (_b_z + beta * k1 * zeta + gamma) * (_c_z + beta * k2 * zeta + gamma) * alpha + L1_zeta * alpha**2 + u) * _Z_t

_D12 = (_a_z + beta * _s1_z + gamma) * (_b_z + beta * _s2_z + gamma) * alpha * beta * _zw_z * _s3_t

_D2 = ZH_zeta * (_TL_t + zeta**n * _TM_t + zeta**(2*n) * _TH_t)

D = _D0 + (_D11 - _D12) - _D2

F = D + v * _A_t + v**2 * _B_t + v**3 * _C_t + v**4 * _s1_t + v**5 * _s2_t

E = -_r0 + v * _a_z + v**2 * _b_z + v**3 * _c_z + v**4 * _s1_z + v**5 * _s2_z + u * _zw_z 

# u * zeta * _Wz_omega_t - questionable
print((_Wz_t + u * _Wz_omega_t) * tau)
print(zeta * _Wz_t + u * zeta * _Wz_omega_t + F - E)

assert (_Wz_t + u * _Wz_omega_t) * tau == zeta * _Wz_t + u * zeta * _Wz_omega_t + F - E 

99
58


AssertionError: 